In [1]:
import numpy as np

In [2]:
from src.preprocessing.data_utils import load_preprocessed_data
from src.preprocessing.data_utils import save_preprocessed_data
from src.preprocessing.data_preprocessing import generate_note_with_instrument_labels
from src.preprocessing.data_preprocessing import calculate_cqt_stats
from src.preprocessing.data_preprocessing import process_files_using_cqt
from src.preprocessing.data_preprocessing import calculate_stft_stats
from src.preprocessing.data_preprocessing import process_files_using_stft

In [7]:
train_data_dir = "../Data/processed/final_data/cqt_bins_24"
train_labels_dir = "../Data/processed/final_data/cqt_bins_24"

In [8]:
train_mean, train_std_dev = calculate_stft_stats(train_data_dir, data_size=1)
X_train, y_train = process_files_using_stft(train_data_dir, train_labels_dir, generate_note_with_instrument_labels, mean=train_mean, std=train_std_dev, data_size=1)

Calculating STFT stats: 2it [00:00, ?it/s]                   


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [12]:
def transform_spectrogram_row(data, row_index, num_rows=5):
    """
    Transforms a specified spectrogram row into a multi-row image by combining it with its neighboring rows.
    
    Parameters:
    data (np.array): Input data (2D array where each row is a time window of the spectrogram)
    row_index (int): Index of the row to be transformed
    num_rows (int): Total number of rows in the resulting image (including the target row and its neighbors)

    Returns:
    np.array: Transformed image (2D array)
    """
    if num_rows % 2 == 0:
        raise ValueError("num_rows must be an odd number.")

    middle_index = num_rows // 2
    num_columns = data.shape[1]  # Number of columns in the input data
    transformed_row = np.zeros((num_rows, num_columns))

    for i in range(num_rows):
        current_row_index = row_index + i - middle_index
        if 0 <= current_row_index < len(data):
            transformed_row[i, :] = data[current_row_index]
        else:
            # If the current_row_index is out of bounds, it remains filled with zeros.
            pass

    return transformed_row

In [ ]:
print(X_train_transformed.shape)

In [7]:
# Count how many 0 and 1 in y
total = 0
print(len(y_train))
for i in range(len(y_train)):
    total += np.sum(y_train[i])
    if i % 10**6 == 0:
        print(total)
print(total)

1
22822
22822


In [9]:
save_preprocessed_data(X_train, y_train, "../Data/test_tmp/saved", "train.h5")

Starting to save preprocessed data...
Data successfully saved to ../Data/test_tmp/saved\train.h5


In [10]:
X_new, y_new = load_preprocessed_data("../Data/test_tmp/saved", "train.h5")

In [11]:
# Count how many 0 and 1 in y_new
total = 0
print(len(y_new))
for i in range(len(y_new)):
    total += np.sum(y_new[i])
    if i % 10**6 == 0:
        print(total)
print(total)

1
22822
22822


In [9]:
X, y = load_preprocessed_data("../Data/processed/final_data/cqt_bins_24", "train.h5")

In [10]:
X[0].shape

(168,)

In [14]:
X_train_transformed = transform_spectrogram_row(X, 0, num_rows=11)
print(X_train_transformed.shape)

(11, 168)


In [7]:
# Count how many 0 and 1 in y
total = 0
print(len(y))
for i in range(len(y)):
    total += np.sum(y[i])
    if i % 10**6 == 0:
        print(total)
print(total)

14732800
0
1539574
2818211
3841920
5363460
6764760
7094242
7439390
8341165
9504707
10793439
11934067
13104221
14516343
15578357
16470462


In [4]:
data = np.load("../Data/musicnet.npz", encoding="latin1", allow_pickle=True)

for file in data.files:
    print(file)
    print(data[file][0].shape)
    print(data[file][0])
    print(data[file][1])
    break

BadZipFile: File is not a zip file

In [ ]:
file = data.files[0]
X = data[file][0]
print(file)
print(len(X))

In [ ]:
file = data.files[0]
labels = data[file][1]
for label in labels:
    print(label[0])
    print(label[1])
    print(label[2][0])
    break

In [ ]:
import pandas as pd
import os

In [ ]:
instruments_set = set()
for file in os.listdir("musicnet/test_labels"):
    data = pd.read_csv("musicnet/test_labels/" + file)
    instruments_set = instruments_set.union(set(data["instrument"].unique()))

for file in os.listdir("musicnet/train_labels"):
    data = pd.read_csv("musicnet/train_labels/" + file)
    instruments_set = instruments_set.union(set(data["instrument"].unique()))
        
print(instruments_set)

In [ ]:
instruments_mapping = {}
for i, instrument in enumerate(instruments_set):
    instruments_mapping[instrument] = i

In [ ]:
instruments_mapping = {
    1: 0,
    7: 1,
    41: 2,
    42: 3,
    43: 4,
    44: 5,
    61: 6,
    69: 7,
    71: 8,
    72: 9,
    74: 10
}

In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

# Function to convert audio waveform to spectrogram
def audio_to_spectrogram(audio, sr, hop_length=512, n_fft=2048):
    # Compute the spectrogram
    S = np.abs(librosa.cqt(audio, sr=sr, hop_length=hop_length, n_fft=n_fft)
    # Convert to decibel scale
    S_db = librosa.amplitude_to_db(S, ref=np.max)
    return S_db

# Example of processing a single audio file from the dataset
def process_single_file(file_id, data):
    audio_data = data[file_id][0]  # Raw audio waveform
    sr = 44100  # Sample rate of the MusicNet dataset is 44.1 kHz

    # Convert the raw audio to a spectrogram
    S_db = audio_to_spectrogram(audio_data, sr)

    # Plot the spectrogram
    plt.figure(figsize=(14, 5))
    librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')
    plt.show()

    return S_db


npz_file_path = "musicnet.npz"
data = np.load(npz_file_path, allow_pickle=True, encoding="latin1")

file_id = '1788' if '1788' in data.files else data.files[0]  # Use the first file if '1788' is not available
spectrogram = process_single_file(file_id, data)

In [ ]:
from Data.preprocessing import DataPreprocessor

preprocessor = DataPreprocessor(transform_type='fft', file_path="../Data/musicnet.npz", metadata_path="../Data/musicnet_metadata.csv")
X, y = preprocessor.load_data()

In [ ]:
import numpy as np

In [ ]:
print(X[0].shape)

In [ ]:
print(y.shape)

In [ ]:
print(y[0].shape)

In [ ]:
max_value = -np.inf
for i in range(len(X)):
    max_value = max(max_value, np.max(X[i]))
print(max_value)